In [7]:
import os
from utils import *
import re
import plotly as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import plotly.express as px
import numpy as np
import tqdm
import plotly.io as pio   
pio.kaleido.scope.mathjax = None

Load datasets

In [8]:
dirc = 'trainset'
_databank = 'crisprbank'
sequences = [os.path.join(dirc,x) for x in sorted(os.listdir(dirc))]
seq_names = [s.split('/')[-1].split('.')[0] for s in sequences]
keys = ['AGG','TGG','GGG','CGG']

(**Step 1**) Function to Identify PAMs and Plot PAM Distributon

In [9]:
def IdentifyPAM(seq):
    PAMs = ['AGG','TGG','GGG','CGG']
    pam_count = [count_pam(seq, pam) for pam in PAMs]
    pam_idx = [find_pam_indices(seq,pam) for pam in PAMs]
    return PAMs, pam_count, pam_idx
print(sequences)
sequences = [read_fasta(s) for s in sequences]
#print(sequences)
marker_style = ['square','circle', 'diamond', 'cross']
subplt_names = ['<b>{}</b><br>(n = {})'.format(seq_names[i], len(str(sequences[i].values()))) for i in range(len(sequences))]
# PLOT
fig = make_subplots(2,4, subplot_titles=subplt_names+['<b>{}'.format(k) for k in keys])
PAM_IDX = []
for i in range(len(sequences)):
    seq = str(sequences[i].values())
    PAMs, pam_count, pam_idx = IdentifyPAM(seq = seq)    
    _df = pd.DataFrame([])
    _df['PAM'] = ['AGG','TGG','GGG','CGG']
    _df['cnts'] = pam_count
    _df = _df.sort_values(by = 'cnts', ascending = False)
    fig.add_trace(go.Bar(x = _df['PAM'], 
                         y = _df['cnts'],
                         name=seq_names[i],
                         marker = dict(color = px.colors.qualitative.G10[i])
                         ), row = 1, col = i+1)
    for k in range(4):
        fig.add_trace(go.Histogram(x = pam_idx[k],
                                 #mode = 'markers',
                                 name = seq_names[i], 
                                 histnorm='probability',
                                 marker = dict(color = px.colors.qualitative.G10[i],
                                               #symbol=marker_style[i],
                                               #size = 11, opacity = 0.3
                                               ),showlegend=False
                                 ), row = 2, col = k+1) 
    PAM_IDX.append(pam_idx)
fig.update_layout(yaxis_title = '<b>PAM Counts')
fig.update_layout(yaxis5_title = '<b>Density')
fig.update_layout(xaxis5_title = '<b>Normalized Index')
fig.update_layout(font = dict(size = 11))
fig.write_image('Result_PAM_Analysis.pdf')
fig.show()

['trainset/cd74.fasta', 'trainset/cd80.fasta', 'trainset/cd9.fasta', 'trainset/clta4.fasta']


(**Step 2**) Target Sequence Extraction

In [10]:
ratio = [0.15,0.25]
start = [int(ratio[0]* len(str(l.values()))) for l in sequences]
end = [int(ratio[1]* len(str(l.values()))) for l in sequences]
print(start)
print(end)
targeted_seq = []
for i, seq in enumerate(sequences):
    seq = str(seq.values())
    seq = seq[start[i]:end[i]]
    targeted_seq.append(seq)
# EXCLUDE "N...N" substrings
targeted_seq = [x.split('N') for x in targeted_seq if len(x.split('N')) > 0]
import itertools
# Merge the list of lists
merged_list = list(itertools.chain(*targeted_seq))
targeted_seq = [x for x in merged_list if len(x)>0]
print(targeted_seq) 
targeted_aa = [translate_dna(x) for x in targeted_seq]
_df = pd.DataFrame([])
_df['TargetedDNA'] = targeted_seq
_df['TargetedAA'] = targeted_aa
_df['Length_DNA'] = [len(x) for x in targeted_seq]
_df['Length_AA'] = [len(x) for x in targeted_aa]
_df = _df.sort_values(by = 'Length_DNA')
_df = _df[_df['Length_DNA'] > 0]
_df.to_csv('Result_TargetedSequences.csv', index = False)

[781, 505, 417, 252]
[1302, 843, 695, 420]
['GAGGCGAGCCGGGGGGTCAGGGTCCCAGATGCACAGGAGGAGAAGCAGGAGCTGTCGGGAAGATCAGAAGCCAGTCATGGATGACCAGCGCGACCTTATCTCCAACAATGAGCAACTGCCCATGCTGGGCCGGCGCCCTGGGGCCCCGGAGAGGT', 'GGATCCTCCTCTGACCTATCCTCCCCACCTCCCACAGCAAGTGCAGCCGCGGAGCCCTGTACACAGGCTTTTCCATCCTGGTGACTCTGCTCCTCGCTGGCCAGGCCACCACCGCCTACTTCCTGTACCAGCAGCAGGGCCGGCTGGACAAACTGACAGTCACCTCCCAGAACCTGCAGCTGGAGAACCTGCGCATGAAGCTTCCCAAGCGTGCGTGCACCCCTACATCCTTGATACCCCCCACCTCCCACCATCCCTCAACTCAG', 'GGTTAAGCTCTCCGGAAGGTTTTAGCAGGCAACTCGAGAGTTTTGCAACTCAAAGTAGGCACACCTTCCTTCCTGAATGACTTTTATTTGTTCCTTTTTCAGGCTGTGAAACTAAATCCACAACCTTTGGAGACCCAGGAACACCCTCCAATCTCTGTGTGTTTTGTAAACATCACTGGAGGGTCTTCTACGTGAGCAATTGGATTGTCATCAGCCCTGCCTGTTTTGCACCTGGGAAGTGCCCTGGTCTTACTTGGGTCCAAATTGTTGGCTTTCACTTTTGACCCTAAGCATCTGAAGCCATGGGCCACACACGGAGGCAGGGAACATCACCATCC', 'TTCCAAAAGCCTCCCATCTGTCATCCCACCCAGACTGCGCGCTTCTAATTCCTCCTACCCCACATGCTGTGCCCAATGAAAAGTATGGTCAGCGAGCGAAGGTTTGCAAGGAGACAGACGAGGGCGAAATTAAGCCAGGCGGCTTCCCTTTAAATCCTCGCAAAGCAGAAGGGCCCCTCACTCT

(**Step 3**) Screening CRISPR

In [11]:
### SCREENING CRISPR
df = [pd.read_csv(os.path.join(_databank,x)) for x in sorted(os.listdir(_databank)) if '.DS_Store' not in x]
df = [d[d['Global_GC']>0.4] for d in df]
df = [d[d['Global_GC']<0.6] for d in df]
df = [d['sgRNA'] for d in df]
df = pd.concat(df)
print(df)
def get_align_score(seq1,seq2):
    # The Most Simple Aligment Criteria of String Bits
    # To-do: More Aligment methods
    alignment_score = sum([1 for a, b in zip(seq1, seq2) if a == b])
    return alignment_score
best_score = 0
SCORES = []
GSTR = []
for gstr in tqdm.tqdm(df):
    global_scores = np.mean([get_align_score(x, gstr) 
                    for x in targeted_seq])
    if global_scores>best_score:
        print(global_scores)
        best_score = global_scores
        GSTR.append(gstr)
        SCORES.append(best_score)
_df = pd.DataFrame([])
_df['mRNA'] = GSTR
_df['SCORE'] = SCORES
_df.to_csv('Result_mRNA.csv', index = False)

0          ACCTACAAGCTCGTCTCTGCAGTGAG
1          TACTACGATAAGCAACTGCTGTGCTG
2          ACGACGTCGATTCAGTGTCTCGCTGC
4          ATCTAGAAGGCAGCGTTGGAAGTCGC
5          CTGCAGCGTGACAAGAGAAAAGGTCA
                      ...            
3545764    AGTATGAATGAGGAGCTCTTGAACGC
3545765    CAGGACGGGACAACTCTATTACACAT
3545766    TCAAGCAGTTACAGTACAGCTTGACG
3545767    ATCTGCACCATGCTATGGATGCTAAT
3545768    TCATGGTAGGTTCGACAGAGAGTCAT
Name: sgRNA, Length: 3407036, dtype: object


  0%|          | 3094/3407036 [00:00<01:50, 30938.11it/s]

6.0
6.6
8.0
9.4
9.6
10.0


  1%|          | 21724/3407036 [00:00<01:49, 30865.26it/s]

10.2


...

 21%|██        | 698996/3407036 [00:22<01:32, 29212.27it/s]

11.0


 35%|███▍      | 1179677/3407036 [00:38<01:12, 30620.36it/s]

11.2


100%|██████████| 3407036/3407036 [01:52<00:00, 30349.84it/s]


(**Step 4**) Merge Final mRNA Molecule

In [12]:
df = pd.read_csv('Result_mRNA.csv')['mRNA']
mRNA = df.tolist()
lcRNA = ''.join(mRNA)
aa_code = translate_dna(lcRNA)
aa_code = aa_code.replace('*','')
print(df)
print(mRNA)
print(lcRNA)
print('----')
print(aa_code)
dirc = 'testset'
sequences = [os.path.join(dirc,x) for x in sorted(os.listdir(dirc))]
seq_names = [s.split('/')[-1].split('.')[0] for s in sequences]
sequences = [read_fasta(s) for s in sequences]
sequences = [list(s.values()) for s in sequences]
alphafold = ['{}:{}'.format(aa_code,s[0]) for s in sequences]
print(seq_names)
print(sequences)
print(alphafold)
df = pd.DataFrame([])
df['alphafold'] = alphafold
df.to_csv('AlphaFold.csv', index = False)

0     ACCTACAAGCTCGTCTCTGCAGTGAG
1     TACTACGATAAGCAACTGCTGTGCTG
2     ATCTAGAAGGCAGCGTTGGAAGTCGC
3     GGAGATGGTCTAGTGATGAGCTTACT
4     GCTGCAGATCTTATGGTAGTGCCTCA
5     GATGAAGCACCAGGGATCTGATCATC
6     TGTGCAAAGTTAGACATCGGCCTACG
7     TGTCCAGCGTATCCGATGTGTCCTGC
8     TGATCGACTCTGCGCATCTTGCACTC
9     TCTTCACATCTAGCGACCGTGCTCCC
10    TAGTAATCACGTGCGATCTGTTTCCC
11    TGTTGAGGTCTACGCATGTGCTTCAC
Name: mRNA, dtype: object
['ACCTACAAGCTCGTCTCTGCAGTGAG', 'TACTACGATAAGCAACTGCTGTGCTG', 'ATCTAGAAGGCAGCGTTGGAAGTCGC', 'GGAGATGGTCTAGTGATGAGCTTACT', 'GCTGCAGATCTTATGGTAGTGCCTCA', 'GATGAAGCACCAGGGATCTGATCATC', 'TGTGCAAAGTTAGACATCGGCCTACG', 'TGTCCAGCGTATCCGATGTGTCCTGC', 'TGATCGACTCTGCGCATCTTGCACTC', 'TCTTCACATCTAGCGACCGTGCTCCC', 'TAGTAATCACGTGCGATCTGTTTCCC', 'TGTTGAGGTCTACGCATGTGCTTCAC']
ACCTACAAGCTCGTCTCTGCAGTGAGTACTACGATAAGCAACTGCTGTGCTGATCTAGAAGGCAGCGTTGGAAGTCGCGGAGATGGTCTAGTGATGAGCTTACTGCTGCAGATCTTATGGTAGTGCCTCAGATGAAGCACCAGGGATCTGATCATCTGTGCAAAGTTAGACATCGGCCTACGTGTCCAGCGTATCCGATGTGTCCTGCTGATCGACT

Use these complexes as input of AlphaFold 2 in ColabFold at: https://colab.research.google.com/github/sokrypton/ColabFold/blob/main/AlphaFold2.ipynb#scrollTo=kOblAo-xetgx.

**License**

The source code of ColabFold is licensed under MIT. Additionally, this notebook uses the AlphaFold2 source code and its parameters licensed under Apache 2.0 and CC BY 4.0 respectively.
